# 🚀 Fine-Tuning Gemma 3 270M on TPU

Production-grade fine-tuning using **Hugging Face Transformers + TRL**

---

## ⚙️ Setup

1. **Runtime** → **Change runtime type** → **TPU**
2. Run cells in order

---

## 📦 Step 1: Install Dependencies

In [ ]:
# Install dependencies
#%pip install -q torch tensorboard
#%pip install -q transformers datasets accelerate evaluate trl protobuf sentencepiece peft

# Optional: Flash Attention (for L4/A100)
# %pip install -q flash-attn

print("✅ Dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 7.9 MB/s eta 0:00:00a 0:00:01
✅ Dependencies installed!


## 🔑 Step 2: HuggingFace Login

In [2]:
from google.colab import userdata
from huggingface_hub import login

# Login to Hugging Face (requires accepting Gemma license)
# Get token from: https://huggingface.co/settings/tokens
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

print("✅ Logged in to Hugging Face!")

TimeoutException: Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.

## 💾 Step 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## ⚙️ Step 4: Configuration

In [ ]:
# @title Configuration
base_model = "google/gemma-3-270m-it"  # @param ["google/gemma-3-270m-it","google/gemma-3-1b-it","google/gemma-3-4b-it"]
checkpoint_dir = "/content/drive/MyDrive/gemma3-finetuned"  # @param {type:"string"}
learning_rate = 5e-5  # @param {type:"number"}
num_epochs = 3  # @param {type:"integer"}
max_length = 512  # @param {type:"integer"}

print(f"📋 Configuration:")
print(f"   Model: {base_model}")
print(f"   Output: {checkpoint_dir}")
print(f"   LR: {learning_rate}")
print(f"   Epochs: {num_epochs}")

## 📊 Step 5: Load Dataset

In [ ]:
from datasets import load_dataset
from google.colab import files

# Option A: Upload your own dataset
print("📤 Upload your dataset (JSONL with 'messages' field):")
uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]

# Load and convert dataset
def create_conversation(sample):
    return {"messages": sample["messages"]}

dataset = load_dataset("json", data_files=dataset_path, split="train")
dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

print(f"\n📊 Dataset loaded:")
print(f"   Train: {len(dataset['train'])} samples")
print(f"   Test: {len(dataset['test'])} samples")
print(f"\n📋 Sample:")
print(dataset["train"][0])

## 🔥 Step 6: Load Model & Tokenizer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model with official Google settings
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="eager"  # Google recommended
)

tokenizer = AutoTokenizer.from_pretrained(base_model)

print(f"✅ Model loaded!")
print(f"   Device: {model.device}")
print(f"   DType: {model.dtype}")

## 🎯 Step 7: Setup LoRA (Optional)

In [ ]:
# Optional: Apply LoRA for memory-efficient training
USE_LORA = True  # @param {type:"boolean"}

if USE_LORA:
    from peft import get_peft_model, LoraConfig, TaskType
    
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )
    
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
else:
    print("📝 Full model fine-tuning (no LoRA)")

## 🚀 Step 8: Training

In [ ]:
from trl import SFTTrainer, SFTConfig

# Training config (Official Google pattern)
torch_dtype = model.dtype

args = SFTConfig(
    output_dir=checkpoint_dir,
    max_length=max_length,
    packing=False,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=4,
    gradient_checkpointing=False,  # Caching incompatible
    optim="adamw_torch_fused",
    logging_steps=1,
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=learning_rate,
    fp16=True if torch_dtype == torch.float16 else False,
    bf16=True if torch_dtype == torch.bfloat16 else False,
    lr_scheduler_type="constant",
    push_to_hub=False,
    report_to="tensorboard",
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": True,
    }
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
)

print("✅ Trainer ready!")

In [ ]:
# Start training
print("🚀 Starting training...")
trainer.train()

# Save final model
trainer.save_model()
print(f"\n✅ Training complete! Model saved to: {checkpoint_dir}")

## 📈 Step 9: Plot Training Loss

In [ ]:
import matplotlib.pyplot as plt

# Extract losses
log_history = trainer.state.log_history
train_losses = [log["loss"] for log in log_history if "loss" in log]
epoch_train = [log["epoch"] for log in log_history if "loss" in log]
eval_losses = [log["eval_loss"] for log in log_history if "eval_loss" in log]
epoch_eval = [log["epoch"] for log in log_history if "eval_loss" in log]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(epoch_train, train_losses, label="Training Loss")
plt.plot(epoch_eval, eval_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

## 🧪 Step 10: Test Inference

In [ ]:
# Test the fine-tuned model
test_prompt = "Hello, how are you?"  # @param {type:"string"}

messages = [{"role": "user", "content": test_prompt}]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
inputs = inputs.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"📝 Prompt: {test_prompt}")
print(f"\n🤖 Response:\n{response}")

---

## ✅ Done!

Your fine-tuned model is saved at: `{checkpoint_dir}`

### Next Steps:
1. Download model from Google Drive
2. Convert to GGUF for Ollama/LM Studio
3. Deploy to production